In [1]:
import gzip
import json
import pandas as pd
import os
import multiprocessing
from functools import reduce, partial
from vaderSentiment.vaderSentiment import sentiment as vaderSentiment

In [2]:
tweets = []
with gzip.open("./olympics/2016-08-02-14.gz") as f:
    for line in f.readlines():
        tweets.append(json.loads(line))

dataset = pd.DataFrame(tweets)

In [3]:
# Create additional columns
dataset['screen_name'] = dataset.user.map(lambda x: x['screen_name'].lower())
dataset['name'] = dataset.user.map(lambda x: x['name'].lower())
dataset['statuses_count'] = dataset.user.map(lambda x: x['statuses_count'])
dataset['followers_count'] = dataset.user.map(lambda x: x['followers_count'])
dataset['text_lower'] = dataset.text.apply(lambda x: x.lower())

In [85]:
corpus = dataset['text_lower']
sentiments = {}

def get_sentiment(scores):
    positive_value = scores['pos']
    negative_value = scores['neg']
    neutral_value = scores['neu']
    
    values = [positive_value, negative_value, neutral_value]
    
    if positive_value == max(values): return "positive"
    if negative_value == max(values): return "negative"
    if neutral_value == max(values): 
        if positive_value > 0.25 or negative_value > 0.25:
            return "positive" if positive_value > negative_value else "negative"
    
def analyse_sentiment(corpus):
    for index, row in corpus.iteritems():
        scores = vaderSentiment(row.encode('utf-8'))
        sentiments[row] = get_sentiment(scores)
        
analyse_sentiment(corpus)

positive_tweets = sum(1 for x in sentiments.values() if x == "positive")
negative_tweets = sum(1 for x in sentiments.values() if x == "negative")

#sentiments

#{k: v for k, v in sentiments.iteritems() if v == "negative"}

#len(corpus)
#positive_tweets / float(len(corpus))
#negative_tweets / float(len(corpus))

0.058936272160996644